In [1]:
import numpy as np
import pandas as pd
from datasets import load_dataset

import torch
from transformers import AutoTokenizer, AutoModel, BertTokenizer, AutoModelForSequenceClassification, pipeline, trainer

tokenizer = AutoTokenizer.from_pretrained('ernie-1.0')
model = AutoModelForSequenceClassification.from_pretrained('ernie-1.0')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
max_len = 200

# raw_datasets = load_dataset("imdb")

Some weights of the model checkpoint at ernie-1.0 were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ernie-1.0 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRA

In [36]:
def tokenize_function(examples):
    return tokenizer(examples['text'], max_length=200, padding='max_length', return_tensors = 'pt')

# tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
# small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
# small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [4]:
chn_train = load_dataset('seamew/ChnSentiCorp')
# BATCH=32
# MAX_SEQLEN=300
# LR=5e-5
# EPOCH=10

Downloading:   0%|          | 0.00/1.88k [00:00<?, ?B/s]

Using custom data configuration default


Downloading:   0%|          | 0.00/3.03M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/376k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/371k [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset chn_senti_corp downloaded and prepared to C:\Users\ZK Hao\.cache\huggingface\datasets\seamew___chn_senti_corp\default\0.0.0\1f242195a37831906957a11a2985a4329167e60657c07dc95ebe266c03fdfb85. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [77]:
def process_data(data):
    # max length is about 306..., truncation and padding are needed
    return tokenizer(data["text"], max_length=300, padding='max_length', truncation=True)


In [78]:
tokenized_datasets = chn_train.map(process_data)

  0%|          | 0/9600 [00:00<?, ?ex/s]

  0%|          | 0/1200 [00:00<?, ?ex/s]

  0%|          | 0/1200 [00:00<?, ?ex/s]

In [103]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42)
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42)

Loading cached shuffled indices for dataset at C:\Users\ZK Hao\.cache\huggingface\datasets\seamew___chn_senti_corp\default\0.0.0\1f242195a37831906957a11a2985a4329167e60657c07dc95ebe266c03fdfb85\cache-b65296123fc3f9f0.arrow
Loading cached shuffled indices for dataset at C:\Users\ZK Hao\.cache\huggingface\datasets\seamew___chn_senti_corp\default\0.0.0\1f242195a37831906957a11a2985a4329167e60657c07dc95ebe266c03fdfb85\cache-2e6f9afa5e3dd721.arrow


In [106]:
from transformers import Trainer, TrainingArguments

model = AutoModelForSequenceClassification.from_pretrained('ernie-1.0', num_labels=2)
training_args = TrainingArguments('finetune_trainer',
                                  evaluation_strategy='epoch',
                                  num_train_epochs=5,
                                  per_device_train_batch_size=4,
                                  per_device_eval_batch_size=4,
                                  metric_for_best_model="accuracy" # check???
                                  )

loading configuration file ernie-1.0\config.json
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "relu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 513,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.11.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 18000
}

loading weights file ernie-1.0\pytorch_model.bin
Some weights of the model checkpoint at ernie-1.0 were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequ

In [104]:
from datasets import load_metric
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


In [105]:
torch.cuda.empty_cache()
model.to(device)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 200
  Num Epochs = 5
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 500


Step,Training Loss


Saving model checkpoint to finetune_trainer\checkpoint-500
Configuration saved in finetune_trainer\checkpoint-500\config.json
Model weights saved in finetune_trainer\checkpoint-500\pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=500, training_loss=0.17784828186035156, metrics={'train_runtime': 113.891, 'train_samples_per_second': 8.78, 'train_steps_per_second': 4.39, 'total_flos': 154166634000000.0, 'train_loss': 0.17784828186035156, 'epoch': 5.0})

In [108]:
torch.cuda.empty_cache()
new_model = AutoModelForSequenceClassification.from_pretrained("finetune_trainer/checkpoint-500")
new_model.to(device)

evaluator = Trainer(
    model=new_model,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
    args=training_args
)


Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

In [114]:
evaluator.evaluate()


loading configuration file finetune_trainer/checkpoint-500\config.json
Model config BertConfig {
  "_name_or_path": "ernie-1.0",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "relu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 513,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.11.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 18000
}

loading weights file finetune_trainer/checkpoint-500\pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initia

In [115]:
# best_run = trainer.hyperparameter_search(n_trials=10, direction="maximize")

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 200
  Batch size = 4


{'eval_loss': 0.4616807699203491,
 'eval_accuracy': 0.94,
 'eval_runtime': 12.9227,
 'eval_samples_per_second': 15.477,
 'eval_steps_per_second': 3.869}

In [ ]:
# for n, v in best_run.hyperparameters.items():
#     setattr(trainer.args, n, v)
#
# trainer.train()

Try for other models
Is it possible to label the text with votes from different models?
Or, only judge whether the emotion is extreme on average
Or, predict hotness with the # of extreme emotion

Or, try to label the dan mu with emotion discovering?